In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [5]:
columns = ['Area', 'Perimeter', 'Compactness', 'KernelLength', 'KernelWidth',
           'AsymmetryCoeff', 'KernelGrooveLength', 'Class']

In [6]:
df = pd.read_csv('seeds_dataset.txt', sep='\s+', names=columns)


In [7]:
df

,Area,Perimeter,Compactness,KernelLength,KernelWidth,AsymmetryCoeff,KernelGrooveLength,Class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [8]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values - 1

In [9]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
X = np.expand_dims(X, axis=-1)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense

inputs = Input(shape=(7, 1))
x = Conv1D(16, kernel_size=3, activation='relu')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1, verbose=1, callbacks=[early_stopping])

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3313 - loss: 1.1719 - val_accuracy: 0.2353 - val_loss: 1.0730
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5618 - loss: 1.0069 - val_accuracy: 0.8824 - val_loss: 0.8661
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7704 - loss: 0.8887 - val_accuracy: 0.8824 - val_loss: 0.7068
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7854 - loss: 0.7905 - val_accuracy: 0.9412 - val_loss: 0.5736
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7332 - loss: 0.7280 - val_accuracy: 0.9412 - val_loss: 0.4647
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8371 - loss: 0.5954 - val_accuracy: 1.0000 - val_loss: 0.3784
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7936 - loss: 0.5557 - val_accuracy: 1.0000 - val_loss: 0.3258
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8446 - loss: 0.4750 - val_accuracy: 1.0000 - val_loss

In [15]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Sequential 모델 정확도: {acc*100:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8631 - loss: 0.2665
Sequential 모델 정확도: 85.71%
